<div align ="right">Thomas Jefferson University <b>COMP 103</b>: Data Analysis and Visualization</div>

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats

# Comparing means with ANOVA

The next means comparison strategy we will investigate is the ANOVA, or ANalysis Of VAriance. This is a means comparison strategy that is often used in the life sciences. The name can be a bit confusing, as this test detects differences among means by calculating the ratios between different variance components - specifically the ratio of the variance between samples (which corresponds to the variance among species, experimental treatments, localities, or whatever else we may be comparing) to the variance within samples (the error or residual variance, variability in the data that cannot be attributed to our experimental factors).  

Similarly to how the the t-test assigned statistical significance by calculating a t-statistic that is then compared to a table of t-values, the ANOVA assigns statistical significance by calculating an **F-statistic** that is compared to a table of F-values. Most simply this can be represented as:

F = <u>between sample variance</u>   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;within sample variance

In this notebook we will compare three different types of ANOVA:
* In a **one-way ANOVA**, we will compare multiple sample means
* In a **two-way ANOVA**, we will compare sample means that are affected by more than one experimental factor
* In a **randomized block ANOVA**, which we will just touch on briefly, we will look at how random factors can be analyzed


**Note 1** ANOVAs can get very complicated very quickly! ANOVAs are calculated differently when sample sizes between groups are different or the same, or when the factors included in an ANOVA are fixed (meaningful beyond the bounds of the experiment being considered, such as level of fertilizer applied, or sex of the study organism) or random (meaning factors that might be expected to vary in unpredictable ways, such as different cages in which lab mice are housed, or fields in which plants are grown). Understanding these differences is very important and planning an experiment that requires ANOVA should be done in consultation with a statistician or an experienced practitioner. 

**Note 2** Many of the resources you will find online about using ANOVA will assume that you are using R statistical software. Running an ANOVA in R will have a different syntax than in python, but the fundamentals are very similar. 


# Comparing multiple means with one-way ANOVA

The simplest application of ANOVA is to compare three or more means to one another. If a two-sample t-test asks 'Does sample A differ from sample B?' a one way ANOVA asks 'Do samples A, B, C, ... differ from one another?' 

To say the same thing a different way, the null hypothesis of an ANOVA is that our sample groups are drawn from the same distribution, and are therefore samples of a population with the same mean and variance. The alternative hypothesis is that our samples are drawn from populations with different means.  

Let's load up a dataset. In the `data` folder is a file called `test_aves.csv`. In the code window below, load that dataset up and print out the header to make sure you have it. 

The data set consists of 1000 average exam scores in three areas: calculus, chemistry, and biology.

In [ ]:
test_aves = pd.read_csv('data/test_aves.csv', usecols=['calc', 'chem', 'biol'])   ## FIX SUPPRESS INDEX
test_aves.head()

Hopefully you see the data in front of you. (If you have peeked at the data you might see that we are ignoring some columns ... we will come back to them with the more complex ANOVAs below right now we are going to pretend that those columns don't exist.) 

First things first, before we start comparing means, let's explore the data. ANOVA has three major assumptions, which are fundamentally the same as the assumptions of regression 1) that the data points are independent of one another 2) that the samples are drawn from a normal distribution, and 3) that the groups being compared have similar variances. Assume that the scores are a random collection of scores from each university, such that each data series in the dataframe is independent, or in other words that the rows don't mean anything. 

Assumption 1) is really a question of experimental design. To start, we will assume that each of these observations is simply a list of the 1000 most recent test averages in each of these disciplines, such that the rows do not represent individual students. That would make things a little trickier, as each student's scores would violate the assumption of independence.

Assumption 2) the normality assumption, is trickier and often mis-stated (try googling it, you will see). An immediate thought might be that we can test this by checking to see if the data is distributed normally, but let's think about that for a second. What if the average test score in each subject was in fact very different? We might expect to see three different curves centered around the average scores for each subject. This would not be a normal distribution of the data! The actual assumption of normality is that the *residuals* are normally distributed. In the case of a one-way ANOVA, that means the differences between the observed data in each group and the mean of each group should themselves be normally distributed.

Assumption 3) is relatively straightforward. Each group must have similar variances. This compares to the regression assumption that variance in y stays constant as the value of x increases, but this is now translated into a situation in which we are comparing different discrete groups. 



### Exercise 1. 

In the code window below, take a pass at using the tools we have learned so far to demonstrate that each of these three conditions is met. You can use visualizations or other statistical tests to do so. You may want to compare the variance directly, plot the data with a histogram, or use the Q-Q plot introduced in the t-test section. 


In [ ]:
###
### Your code here
###

In this case a p value of < 0.05 would indicate that we were observing a significant deviation from normality. We observe that only for kurtosis of the combined data. These are very powerful tests of deviations from normality.

The most important data are actually in our first two results, from which we can see that the standard deviations for each group are approximately equal and each group is an approximately normal distribution.

Notice that none of these test directly answered the question of whether the residuals were normally distributed. In order to do that we would first have to run a statistical test in order to generate the residuals. 

In a future iteration we will introduce the conept of a Q-Q plot here, you can read about it to see what that looks like ... https://en.wikipedia.org/wiki/Q–Q_plot

### Running a one-way ANOVA

In a one-way ANOVA we just want to compare groups means. In some ways this is the last of the "simple" tests. 
To run a one-way anova within stats we can use the `stats.f_oneway(group0, group1 ... groupN)`function from the stats library. Each `group` needs to be a data series. 


### Exercise 2. 

Using the code above. Run a one-way anova comparing calc, chem, and biology scores from the `test_aves` data.  

Create a boxplot of the means to compare this data to. What do we see?

In [ ]:
stats.f_oneway(test_aves['calc'], test_aves['chem'], test_aves['biol'])

### Question 1

As best you can interpret the output above. What does it tell you? What doesn't it tell you? Does it match what you expect from the boxplot output?

 * 
 *
 *

## Post hoc tests with one-way ANOVA
I think you will agree that the results that we got from the one-way ANOVA were kind of underwhelming - we fot an F statistic and a (highly significant) p value. But we don't get any statements about *what* is significant or what difference between groups might be driving the observed difference. This simple p value just tells us that the groups are not drawn from the same distribution. 

It would be useful to know more. To determine which groups are different from one another, we need to do what is known as a post-hoc test. The test we would use in this circumstance is the Tukey HSD test, where HSD means Highly Significant Differences. 

The syntax for this test is very simple and follows the model of the one-way ANOVA:  `stats.tukey_hsd(group0, group1 ... groupN)`.

In [ ]:
print(stats.tukey_hsd(test_aves['calc'], test_aves['chem'], test_aves['biol']))

This table gives us the comparison between each group, so `Comparison (0 - 1)` refers to a comparison between group 0 and group 1. 

### Question 2 
What is this table telling us in terms of biology, chemistry, and calculus scores. How would this table differ if chemistry and biology scores were not different from one another? 

 * 
 * 
 * 

## ANOVA tables and `statsmodels`
Reporting an ANOVA is more complicated than just F, and p, however. ANOVA results are generally reported in a stereotyped format known as an ANOVA table. One problem with the stats library is that it's reporting functions are very rudimentary and we aren't getting all of the information that we would want to get. 

We also don't have any way to access additional information from the ANOVA analysis - such as how the variances in the test were partitioned - since the `stats` anova approach is so simple. 

In order to move forward into more complicated analyses we need to go to a different, more sophisticated tool. The one that is going to give us the most flexibility is the `statsmodels` library which we will see improves reporting AND gives us the flexibility to do different types of ANOVA. 

`statsmodels` is, as the name implies, a models-based approach. In essence what we are going to do is to create a statistical model which is fit to the data. 

An important thing about statsmodels is that we can't use it on just any data structure. We need the datat to be in a fully-expanded of long formal. Our data is pretty simple so we can use the handy pandas function `melt` to do this conversion. 


In [ ]:
import statsmodels as sm
from statsmodels.formula.api import ols         # ols = Ordinary Least Squares regression
from statsmodels.stats.anova import anova_lm    # anova_lm = organizes a linear model into
                                                ## an ANOVA format
test_aves_long = test_aves.melt(var_name = "subject", value_name = "average")
print(test_aves_long)



### Question 3
In your own words, what did `melt()` do? What kind of data does `melt()` work on? What is `melt()` and what library does it belong to? What are the arguments?

 * 
 * 
 * 





### Running a one-way ANOVA in statsmodels and making an ANOVA table 

Now that the data is in a long format we can apply the model. 

The model uses the tilde as an operator. This follows the pattern of the R language, and in fact, the statsmodels library is a way of running 'R-type' statistical models in python. The first value in the model is y - which in our 'melted' data set is the column 'average'. Our second value is our x value, 'subject' The C operator lets the model know that this is categorical data, and not a continuous x value (for say, performing a regression). 

This is what that syntax looks like, go ahead and run it: 

In [ ]:
model = ols('average ~ C(subject)', data = test_aves_long).fit()
one_way_table = anova_lm(model, typ=2)
one_way_table

### Question 3

Did this give us the same result as before? How do you know? What else is being reported in this table. Why might this be useful? Which do you prefer?

 * 
 * 
 * 

Why does it make sense that the data would be "melted" to get this result?

 * 
 * 
 * 



### post hoc tests with statsmodels

We can run posthoc tests with statsmodels as well. The syntax is very similar.

In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

pairwise = pairwise_tukeyhsd(test_aves_long['average'],test_aves_long['subject'] )
print(pairwise)

### Question 4

How does this compare to our previous results? Is this a better approach?

 * 
 * 
 * 




# Two-way analysis of variance

You have seen that statsmodels gives you much better and interpretable results from an ANOVA than the base stats package. It's real power, however, is when we want to run more complicated models. The next step for us means running ANOVAs where we have more than one variable for categorizing the data. 

Let's read in the full dataset below. We will drop our initial assumption that all of this data came from the same population and see that we now have scores classified by student sex and by the university at which the data was collected. 



In [ ]:
test_data = pd.read_csv('data/test_aves.csv')
test_data.head()

The point of a two way anova is to attribute variation to two variables at the same time. Let's look at the situation where we compare `subject` and `sex`. Each of these *factors* or predictor variables is going to act like an individual 'x' variable or grouping factor. We still have a single 'y'  variable in our scores. 

We are going to ask our test if there is a difference in subject (which we already know there is) and if there is a difference in sex (which we don't know). We also have to include a term called an **interaction** factor. An interaction occurs when the effect of two factors are not independent of one another. The interaction factor in the model tests for that independence. If the factors are not independent, post-hoc tests are not possible. We will return to interactions later. 

The first thing we need to do is remelt the data down into a long form (notice how we identify that there are multiple categorical variables associated with each data point):

In [ ]:
test_data_long = test_data.melt(id_vars = ['univ', 'sex'],var_name = "subject", value_name = "average")
test_data_long.head()

The functions that we use to run a two-way ANOVA model are the same as the ones that we use to run a one way, we will just set up our equation differently. 

The `*` operator means that we will use the factors `subject` and `sex` as predictor variables in our model and that we will also consider the interaction between these two variables. We could have used the syntax:
```
ols('average ~ C(subject) + C(sex) + C(subject):C(sex)', data = test_data_long).fit()
```
and we would get the exact same result. The `:` represents the interaction as you will see in the results table below. We will print the ANOVA table and a results summary from the linear regression model:

In [ ]:
model2way = ols('average ~ C(subject)*C(sex)', data = test_data_long).fit()
two_way_table = anova_lm(model2way, typ=2)
two_way_table

In [ ]:
print(model2way.summary())

### Question 5
Looking just at the ANOVA table, which of the three model factors are significant at the p<0.05 level? How would you interpret this result in common language. Do we need to do any additional post_hoc testing to understand this result?

 * 
 * 
 * 


Looking at the summary - what does the R^2 value mean at the top of the regression results?
What does your intuition tell you that the 'coefficient' variables mean in the lower part of the summary table? How about `P>|t|`?


 * 
 * 
 * 



## Wrap up and exercises

Since there is no interaction between sex and subject, if we wanted to run a post-hoc test on this data we could run our Tukey test again. If there is an interaction, that wouldn't work. We will demonstrate this with Exercise 1. 

### Exercise 1 
Run the code window below. Comment the code and explain step by step what is happening. Explain the ols model used. Assume that you are looking at two plant species A and B that are receiving or not receiving a growth treatment. Interpret the results. Then make a line graph that shows  the average values of the species in each treatment combination. Interpret this graph. Why does this graph show an **interaction**

In [ ]:
df_int =  pd.DataFrame(
    {'species': ['A','A','A','A','B','B','B','B'],
    'condition':  ['X','C','X','C','X','C','X','C'],
    'measurement':  [42, 35, 41, 34, 33, 36, 35, 34]})
df_int


model_int = ols('measurement ~ C(species)*C(condition)', data = df_int).fit()
two_way_table = anova_lm(model_int, typ=2)
two_way_table

### Exercise 1 answer here:

 * 
 * 
 * 

### Exercise 2:

Re-analyze the test average data. Plot histograms of the data by university to get a sense of what is going on before you start. Do a two-way anova in which you consider the factors university and subject. Make an ANOVA table, and present the data summary. Do you see an interaction? Do a post-hoc test on the university factor, and interpret it. 


In [ ]:
#
## your code here
#

![TJU logo image](images/TJU_logo_image.png "TJU logo image")